<font size="+3"><strong>Predicting Price with Location</strong></font>

In [ ]:
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline, make_pipeline

warnings.simplefilter(action="ignore", category=FutureWarning)

# Prepare Data

## Import

In [ ]:
def wrangle(filepath):
    #import_csv
    df = pd.read_csv(filepath,index_col=[0])

    #subset to propertes in '"Capital Federal"'
    mask_ba= df["place_with_parent_names"].str.contains("Bogotá D.C")

    #subset for  '"apartment"'
    mask_apt=df["property_type"] == "house"

    #subset where '"price_aprox_usd"' < 400,000
    mask_price= df["price_aprox_usd"] < 400_000

    df= df[mask_ba & mask_apt & mask_price]

    #Remove the outliers by '"surface_covered_in_m2"'
    low, high= df["surface_covered_in_m2"].quantile([0.1,0.9])
    mask_area =df["surface_covered_in_m2"].between(low, high)
    df=df[mask_area]

    return df

In [ ]:
# create a DataFrame `frame1` from the CSV file `colombia-real-estate-1.csv`
frame1 = wrangle("/content/colombia-real-estate-1.csv")
print(frame1.info())
frame1.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504 entries, 132 to 2993
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   operation                   504 non-null    object 
 1   property_type               504 non-null    object 
 2   place_with_parent_names     504 non-null    object 
 3   lat-lon                     504 non-null    object 
 4   price                       504 non-null    float64
 5   currency                    504 non-null    object 
 6   price_aprox_local_currency  504 non-null    float64
 7   price_aprox_usd             504 non-null    float64
 8   surface_total_in_m2         157 non-null    float64
 9   surface_covered_in_m2       504 non-null    float64
 10  price_usd_per_m2            157 non-null    float64
 11  price_per_m2                504 non-null    float64
 12  floor                       333 non-null    float64
 13  rooms                       489 

,operation,property_type,place_with_parent_names,lat-lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url
132,sell,house,|Colombia|Bogotá D.C|Usaquén|,"4.724,-74.023",6.300000e+08,COP,6.349309e+08,212721.41,NaN,135.0,NaN,4.666667e+06,NaN,3.0,NaN,http://usaquen-bogota-d-c.properati.com.co/jm3...
134,sell,house,|Colombia|Bogotá D.C|Los Mártires|,"4.618,-74.081",5.000000e+08,COP,5.039134e+08,168826.52,NaN,240.0,NaN,2.083333e+06,NaN,3.0,NaN,http://los-martires-bogota-d-c.properati.com.c...
138,sell,house,|Colombia|Bogotá D.C|Suba|,"4.729,-74.053",3.950000e+08,COP,3.980916e+08,133372.95,NaN,134.0,NaN,2.947761e+06,NaN,3.0,NaN,http://suba-bogota-d-c.properati.com.co/jm4g_v...
141,sell,house,|Colombia|Bogotá D.C|Usaquén|,"4.667,-74.05",1.100000e+09,COP,1.108609e+09,371418.34,NaN,143.0,NaN,7.692308e+06,NaN,3.0,NaN,http://usaquen-bogota-d-c.properati.com.co/jm4...
143,sell,house,|Colombia|Bogotá D.C|Suba|,"4.702,-74.06",7.000000e+08,COP,7.054787e+08,236357.12,NaN,248.0,NaN,2.822581e+06,2.0,4.0,NaN,http://suba-bogota-d-c.properati.com.co/jm5d_v...


In [ ]:
#split the '"lat-lon"'
    df[["lat" ,"lon"]]= df["lat-lon"].str.split(",", expand= True).astype(float)
    df.drop(columns="lat-lon", inplace = True)

In [ ]:
def wrangle(filepath):
    #import_csv
    df = pd.read_csv(filepath,index_col=[0])

    #subset to propertes in '"Capital Federal"'
    mask_ba= df["place_with_parent_names"].str.contains("Bogotá D.C")

    #subset for  '"apartment"'
    mask_apt=df["property_type"] == "house"

    #subset where '"price_aprox_usd"' < 400,000
    mask_price= df["price_aprox_usd"] < 400_000

    df= df[mask_ba & mask_apt & mask_price]

    #Remove the outliers by '"surface_covered_in_m2"'
    low, high= df["surface_covered_in_m2"].quantile([0.1,0.9])
    mask_area =df["surface_covered_in_m2"].between(low, high)
    df=df[mask_area]

    #split the '"lat-lon"'
    df[["lat" ,"lon"]]= df["lat-lon"].str.split(",", expand= True).astype(float)
    df.drop(columns="lat-lon", inplace = True)

    return df

In [ ]:
frame1 = wrangle("/content/colombia-real-estate-1.csv")
print(frame1.info())
frame1.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504 entries, 132 to 2993
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   operation                   504 non-null    object 
 1   property_type               504 non-null    object 
 2   place_with_parent_names     504 non-null    object 
 3   price                       504 non-null    float64
 4   currency                    504 non-null    object 
 5   price_aprox_local_currency  504 non-null    float64
 6   price_aprox_usd             504 non-null    float64
 7   surface_total_in_m2         157 non-null    float64
 8   surface_covered_in_m2       504 non-null    float64
 9   price_usd_per_m2            157 non-null    float64
 10  price_per_m2                504 non-null    float64
 11  floor                       333 non-null    float64
 12  rooms                       489 non-null    float64
 13  expenses                    0 no

,operation,property_type,place_with_parent_names,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon
132,sell,house,|Colombia|Bogotá D.C|Usaquén|,6.300000e+08,COP,6.349309e+08,212721.41,NaN,135.0,NaN,4.666667e+06,NaN,3.0,NaN,http://usaquen-bogota-d-c.properati.com.co/jm3...,4.724,-74.023
134,sell,house,|Colombia|Bogotá D.C|Los Mártires|,5.000000e+08,COP,5.039134e+08,168826.52,NaN,240.0,NaN,2.083333e+06,NaN,3.0,NaN,http://los-martires-bogota-d-c.properati.com.c...,4.618,-74.081
138,sell,house,|Colombia|Bogotá D.C|Suba|,3.950000e+08,COP,3.980916e+08,133372.95,NaN,134.0,NaN,2.947761e+06,NaN,3.0,NaN,http://suba-bogota-d-c.properati.com.co/jm4g_v...,4.729,-74.053
141,sell,house,|Colombia|Bogotá D.C|Usaquén|,1.100000e+09,COP,1.108609e+09,371418.34,NaN,143.0,NaN,7.692308e+06,NaN,3.0,NaN,http://usaquen-bogota-d-c.properati.com.co/jm4...,4.667,-74.050
143,sell,house,|Colombia|Bogotá D.C|Suba|,7.000000e+08,COP,7.054787e+08,236357.12,NaN,248.0,NaN,2.822581e+06,2.0,4.0,NaN,http://suba-bogota-d-c.properati.com.co/jm5d_v...,4.702,-74.060


In [ ]:
# Create a DataFrame `frame2` from the file `colombia-real-estate-2.csv`
frame2 = wrangle("colombia-real-estate-2.csv")
frame2.head()

,operation,property_type,place_with_parent_names,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon
3000,sell,house,|Colombia|Bogotá D.C|Kennedy|,440000000.0,COP,4.434438e+08,148567.33,161.0,161.0,922.778447,2.732919e+06,NaN,4.0,NaN,http://kennedy-bogota-d-c.properati.com.co/k4o...,4.631,-74.147
3002,sell,house,|Colombia|Bogotá D.C|Rafael Uribe Uribe|,334900000.0,COP,3.375212e+08,113080.00,384.0,131.0,294.479167,2.556489e+06,3.0,10.0,NaN,http://rafael-uribe-uribe-bogota-d-c.properati...,4.585,-74.123
3011,sell,house,|Colombia|Bogotá D.C|Suba|,350000000.0,COP,3.527394e+08,118178.56,112.0,112.0,1055.165714,3.125000e+06,NaN,3.0,NaN,http://suba-bogota-d-c.properati.com.co/k4qt_v...,4.747,-74.062
3014,sell,house,|Colombia|Bogotá D.C|Usaquén|,910000000.0,COP,9.171224e+08,307264.26,NaN,164.0,NaN,5.548780e+06,8.0,3.0,NaN,http://usaquen-bogota-d-c.properati.com.co/k4r...,4.705,-74.044
3015,sell,house,|Colombia|Bogotá D.C|Suba|,980000000.0,COP,9.876703e+08,330899.98,190.0,190.0,1741.578842,5.157895e+06,NaN,2.0,NaN,http://suba-bogota-d-c.properati.com.co/k4rx_v...,4.715,-74.095


In [ ]:
# Combine our DataFrames into single dataframe `df`
df = pd.concat([frame1,frame2], ignore_index=True )
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199 entries, 0 to 1198
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   operation                   1199 non-null   object 
 1   property_type               1199 non-null   object 
 2   place_with_parent_names     1199 non-null   object 
 3   price                       1199 non-null   float64
 4   currency                    1199 non-null   object 
 5   price_aprox_local_currency  1199 non-null   float64
 6   price_aprox_usd             1199 non-null   float64
 7   surface_total_in_m2         431 non-null    float64
 8   surface_covered_in_m2       1199 non-null   float64
 9   price_usd_per_m2            431 non-null    float64
 10  price_per_m2                1199 non-null   float64
 11  floor                       818 non-null    float64
 12  rooms                       1162 non-null   float64
 13  expenses                    1 non

,operation,property_type,place_with_parent_names,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon
0,sell,house,|Colombia|Bogotá D.C|Usaquén|,6.300000e+08,COP,6.349309e+08,212721.41,NaN,135.0,NaN,4.666667e+06,NaN,3.0,NaN,http://usaquen-bogota-d-c.properati.com.co/jm3...,4.724,-74.023
1,sell,house,|Colombia|Bogotá D.C|Los Mártires|,5.000000e+08,COP,5.039134e+08,168826.52,NaN,240.0,NaN,2.083333e+06,NaN,3.0,NaN,http://los-martires-bogota-d-c.properati.com.c...,4.618,-74.081
2,sell,house,|Colombia|Bogotá D.C|Suba|,3.950000e+08,COP,3.980916e+08,133372.95,NaN,134.0,NaN,2.947761e+06,NaN,3.0,NaN,http://suba-bogota-d-c.properati.com.co/jm4g_v...,4.729,-74.053
3,sell,house,|Colombia|Bogotá D.C|Usaquén|,1.100000e+09,COP,1.108609e+09,371418.34,NaN,143.0,NaN,7.692308e+06,NaN,3.0,NaN,http://usaquen-bogota-d-c.properati.com.co/jm4...,4.667,-74.050
4,sell,house,|Colombia|Bogotá D.C|Suba|,7.000000e+08,COP,7.054787e+08,236357.12,NaN,248.0,NaN,2.822581e+06,2.0,4.0,NaN,http://suba-bogota-d-c.properati.com.co/jm5d_v...,4.702,-74.060


## Explore

In [ ]:
# Create a Mapbox scatter plot that shows the location of the houses in `df`
fig = px.scatter_mapbox(
            df, # Our DataFrame
            lat="lat",
            lon="lon",
            width=600, # Width of map
            height=600, # Height of map
            color="price_aprox_usd",
            hover_data=["price_aprox_usd"], # Display price when hovering mouse over house
)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [ ]:
# Create 3D scatter plot
fig = px.scatter_3d(
         df,
         x="lon",
         y="lat",
         z="price_aprox_usd",
         labels={"lon": "longitude", "lat": "latitude", "price_aprox_usd": "price"},
         width=600,
         height=500,
)

# Refine formatting
fig.update_traces(marker={"size": 4, "line": {"width": 2, "color": "DarkSlateGrey"}},
                  selector={"mode": "markers"},
)

# Display figure
fig.show()

## Split

In [ ]:
# Create the feature matrix
features = ["lon", "lat"]
X_train = df[features]
X_train.shape

(1199, 2)

In [ ]:
# Create the target vector
target = "price_aprox_usd"
y_train = df[target]
y_train.shape

(1199,)

# Build Model

## Baseline

In [ ]:
# Calculate the baseline mean absolute error
y_mean= y_train.mean()
y_pred_baseline =[y_mean] * len(y_train)
print("Mean house price:", round(y_mean, 2))
print("Baseline MAE:", mean_absolute_error(y_train, y_pred_baseline))

Mean apt price: 195960.13
Baseline MAE: 72006.54


# Iterate

In [ ]:
# Create a pipeline named model that contains a `SimpleImputer` transformer followed by a `LinearRegression` predictor
model = make_pipeline(
           SimpleImputer(),
           LinearRegression()
)

In [ ]:
model.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('linearregression', LinearRegression())])

## Evaluate

In [ ]:
# Calculate the training mean absolute error
y_pred_training = model.predict(X_train)
mae_training = mean_absolute_error(y_train, y_pred_training)
print("Training MAE:", round(mae_training, 2))

Training MAE: 69170.32


# Results

In [ ]:
# Extract the intercept and coefficients for our model
intercept = model.named_steps["linearregression"].intercept_.round()
coefficients = model.named_steps["linearregression"].coef_.round()
coefficients

array([ 834598., -261361.])

In [ ]:
# Print the equation that our model has determined for predicting
print(
f"price = {intercept} + ({coefficients [0]} * longitude) + ({coefficients[1]} * latitude)"
)

price = 63235589.0 + (834598.0 * longitude) + (-261361.0 * latitude)


In [ ]:
# Create 3D scatter plot
fig = px.scatter_3d(
         df,
         x="lon",
         y="lat",
         z="price_aprox_usd",
         labels={"lon": "longitude", "lat": "latitude", "price_aprox_usd": "price"},
         width=600,
         height=500,
)

# Create x and y coordinates for model representation
x_plane = np.linspace(df["lon"].min(), df["lon"].max(), 10)
y_plane = np.linspace(df["lat"].min(), df["lat"].max(), 10)
xx, yy = np.meshgrid(x_plane, y_plane)

# Use model to predict z coordinates
z_plane = model.predict(pd.DataFrame({"lon": x_plane, "lat": y_plane}))
zz = np.tile(z_plane, (10, 1))

# Add plane to figure
fig.add_trace(go.Surface(x=xx, y=yy, z=zz))

# Refine formatting
fig.update_traces(
         marker={"size": 4, "line": {"width": 2, "color": "DarkSlateGrey"}},
         selector={"mode": "markers"},
)

# Display figure
fig.show()